In [1]:
import pandas as pd
import duckdb

In [2]:
df = pd.read_csv("/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/bangalore/raw/location_poi_data.csv")

In [3]:
all_data = df.copy()
con = duckdb.connect(database='duckdb_gmaps.db', read_only=False)
con.register('all_unique_gmaps_data', all_data)

# create a table from the dataframe so that it persists
con.execute("drop TABLE if exists gmaps_data_eye  ")

con.execute("CREATE TABLE gmaps_data_eye AS (SELECT * FROM all_unique_gmaps_data)")
#con.close()


con.execute("""PRAGMA create_fts_index(gmaps_data_eye, 'id', 'name', stemmer='porter', stopwords='english',
                 ignore='(\\.|[^a-z])+', strip_accents=1, lower=1, overwrite=1)
            """)


In [12]:
import json
json.dumps({
        'competitor_index': 0.20,
        'affluence_index': 0.19,
        'apartments_index': 0.19,
        'fashion_index': 0.19,
        'vibrancy': 0.14,
        'healthcare_index': 0.10,
        'company_index': 0.10,
        'malls_index': 0.05,
        'supermarket_index': 0.04,

    })

'{"competitor_index": 0.2, "affluence_index": 0.19, "apartments_index": 0.19, "fashion_index": 0.19, "vibrancy": 0.14, "healthcare_index": 0.1, "company_index": 0.1, "malls_index": 0.05, "supermarket_index": 0.04}'

In [27]:

b = ['colorbar', 'just herbs',
     'lakme',
     'biotique',
     'maybelline',
     'hudabeauty',
     'lys beauty',
     'beauty bakerie',
     'bobbi brown', 'estee lauder', 'mamaearth', 'myglamm', 'mcaffeine', 'Disguise Cosmetics', 'Lotus Botanicals',
     'Forest Essentials', 'WOW']

In [4]:
b = ["Sugar cosmetics","Nailbox", "Nailashes", "Blown","W for Women", "Biba",  "Aurelia","houseofindya","forever21","FabIndia","Madame",
     "Only","Lakshita","giniandjony", "bornbabies","nykaa","Global Desi","Libas","andindia","Gocolors","Naturals Salon", "Bounce","BBlunt","Plum", ]

In [8]:
b=['colordust nails','The Palette']

In [9]:
all_brands = []
for i in b:
    df2 = con.execute(f"""select * from(SELECT *, fts_main_gmaps_data_eye.match_bm25(
            id, '{i.replace("'", '')}', fields := 'name',conjunctive := 1) AS score FROM gmaps_data_eye) where score>0
        """).fetch_df()
    brands = df2['brand_id']
    all_brands.append(df2)


In [31]:
pd.concat(all_brands).query("brand_id=='cricheroes'")

,id,name,lat,lng,cid,brand_id,brand_name,number_of_votes,rating,place_url,source,category,top_category,type,created_at,score
150,ChIJwZBlSSsTrjsRgbgZLQSLetg,WOW SPORTS,12.93601,77.744583,0x3bae132b496590c1:0xd87a8b042d19b881,cricheroes,cricheroes,161,4.1,https://www.google.com/maps/place/?q=place_id:...,gmaps_v2,school,education,N_A,1687847039,3.765694


In [10]:
brands = list(set(pd.concat(all_brands)[['id', 'name', 'brand_id', 'brand_name', 'score']]['brand_id'].tolist()))
brands = list(filter(lambda x: x != 'N_A', brands))
brands

[]